# Important
This merges and cleans the data putting it into one master CSV 

We handle all computationally heavy manipulation with the data here so that we don't effect the spiders runtime


In [ ]:
import glob
import pandas as pd
import re
import os
from datetime import datetime
from DSPG_Products import Products
from DSPG_Branding import BrandIdentification
from geopy.geocoders import GoogleV3
# Importing Keys
import sys
sys.path.append('Hidden/')
from HiddenKeys import Keys
sys.path.remove('Hidden/')

# Setup
currentDate = str(datetime(datetime.today().year, datetime.today().month, datetime.today().day))[:-8]
# folder_path = currentDate + 'Data'  # the actual folder path
folder_path = 'Data'  # the actual folder path
CSVDataframes = {}  # Dictionary to store CSV dataframes

# Getting the CSV files from the folder path
for file_path in glob.glob(os.path.join(folder_path, '*.csv')):
    file_name = os.path.basename(file_path)
    if "master" in file_name.title().lower():  # Failsafe in case we run it twice and there's already a master CSV
        continue
   
    df = pd.read_csv(file_path)
    name = ' '.join(file_name.title().split()[1:]).split('.')[0]
    contains_skipped = df.astype(str).apply(lambda x: x.str.contains('SKIPPED')).any().any()
    if contains_skipped:
        print("WARNING Skip value found in: ", file_name)    
    CSVDataframes[name] = df
    print("Added", file_name)

print("Files imported")

# Getting what we need from the Products class
product = Products()
MasterDataFrame = product.ProductDataFrames
# Adding some new columns
for df in MasterDataFrame:
    df.insert(len(df.columns)-6, 'Local', None)
    num = len(df.columns)-2
    df.insert(num, 'Store', None)
    df.insert(num, 'Longitude', None)    
    df.insert(num, 'Latitude', None)

# Regular expression pattern to match any of the words to remove and capture the word
pattern = re.compile(r'\b(({})\b)'.format('|'.join(map(lambda x: re.escape(x[1]), product.ProductList))), re.IGNORECASE)

# Combining the dataframes into there respected Master data frames and adding store column
for name, data in CSVDataframes.items():
    # #Fills in the store
    data['Store'] = pattern.sub("", name)
    #Finds the type of product and adds the dataframe to the corresponding master data frame
    productName = [m[0] for m in pattern.findall(name)]
    index = next((index for index, word in product.ProductList if word in productName[0]), None)
    MasterDataFrame[index] = pd.concat([MasterDataFrame[index], data], ignore_index=True)

print("Master files created")
print("Setting and cleaning values")
# Initialize the geocoder with API key
geolocator = GoogleV3(api_key= Keys().geolocator)
brandClean = BrandIdentification()

for indexFrame, dataFrame in enumerate(MasterDataFrame):
    for index, row in dataFrame.iterrows():
        location = geolocator.geocode(str(row['Address']) + " " + str(row['City']) + " " + str(row['State']) + " " + str(row["Zip Code"]))
        if location:
            dataFrame.loc[index, 'Latitude'] = location.latitude
            dataFrame.loc[index,'Longitude'] = location.longitude      
            
        brand, locality = brandClean.determineLocality(row['Brand'], row[0], indexFrame)
        dataFrame.loc[index,'Brand'] = brand
        dataFrame.loc[index,'Local'] = locality
       
print("Finished setting and cleaning values")  

#Saving the data frames
for index, frame in enumerate(MasterDataFrame):
    frame.to_csv(os.path.join(folder_path, "Master " + currentDate + product.ProductList[index][1]+ ".csv"), index=False)
print("Master files exported")
